# BooleAn Networks on the GPU (BANG)

In [ ]:
!pip install bang-gpu
# Project by:
# Paweł Zając, Bartłomiej Parapura, Jan Jagodziński, Mikołaj Czarnecki
# Commisioned by:
# Jakub Zarzycki, Andrzej Mizera
# Supervised by:
# Robert Dąbrowski

## (Probabilistic) Boolean Networks
Probabilistic Boolean Networks are mathematical structures consisting of a set of Boolean variables as well as one or more functions corresponding to each variable and computing its state based on current values of variables.

In the case of deterministic Boolean Networks, each variable has only one function which can depend on any other variable. In the case of Probabilistic Boolean Networks, each variable can have many functions, each of the functions having its associated probability of being chosen as the rule for the update.

Eventually, after a finite amount of time steps the states of these Networks stabilize into just a few constantly recurring values - the sets of these values are called **attractors**.
<div style="text-align: center;">
  <img src="notebook_files/img/title-pbn.png" alt="gre" width="600"/>
</div>

## Visualization of Boolean Networks

 The example on the left below shows a graph of the networks' nodes - if a node's value influenced another node (this means it's present in the node's function), then we draw an edge from the former to the latter.

 The example on the right shows a State Transition Diagram, which directly represent the evolution of the network in time. Given any state, we can trace where the network will end up by following all of its outgoing edges. In this particular example, we can see that there is a single attractor with states 000, 101 and 111.

<div style="text-align: center">
<img src="./notebook_files/img/paper_dep_and_state_graph.png" width="800"/>
</div>

## What are Probabilistic Boolean Networks used for?
Probabilistic Boolean Networks in spite of a relatively simple structure allow to model behaviour of complex systems in discrete time series. One of the most often modelled objects are **gene regulatory networks**, used for modelling evolution of cell in a time period. Nodes of Boolean Networks correspond well to the genes of a cell and attractors are a great representation of final stages of cell evolution.



The final stages could be stem cells or liver cells - knowledge of the attractors of the network can provide insight needed to turn one into the other by controlling which genes are active and which are not.

<div style="text-align: center">
<img src="./notebook_files/img/grn_highres.png" width="400px" height="400px" />
</div>
Source: https://doi.org/10.3389/fncel.2014.00437

## Our contribution

Previous software uses Java and doesn't provide enough features for analyzing long-term network behavior such as attractors. We propose a Python library that contains API for handling, manipulation and analysis of Probabilistic Boolean Networks on GPU.

### Features to implement:
- Loading and parsing of PBNs
- Simulation of PBN traversal on GPU
- Visualization
- Attractor detection
  - Block decomposition
  - Monte Carlo

## Loading PBN from a file

Our library can load files from Systems Biology Markup Language (*.sbml*) format which is an extension of XML and a standard for representing biological models. Alternatively, we can import files in .*pbn* format which was used in previous software modelling PBNs.



Now we can load Probabilistic Boolean Network from this file


In [ ]:
import bang

pbn = bang.load_from_file("examples/example_network.pbn", "assa")

We can also load PBNs from popular sbml format:

In [ ]:
sbml_pbn = bang.load_from_file("examples/example-network.sbml", "sbml")

#source: 
#Giacomantonio CE, Goodhill GJ (2010) 
#A Boolean Model of the Gene Regulatory Network Underlying Mammalian Cortical Area Development. 
#PLOS Computational 

## Simulation
Our Probabilistic Boolean Network is ready to be simulated on GPU!

We can run a simple test that concurrently runs 5 time steps on 512 networks at once and returns their final states. We can choose one of three update types: synchronous, asynchronous where we pick one node and asynchronous where we pick all nodes and choose order randomly.

In [ ]:
pbn.set_states([[True, True, True] for _ in range(512)], reset_history=True)
pbn.simple_steps(4)

for i in range(len(pbn.history_bool)):
    print(pbn.history_bool[i][0])


As a result, we receive the history of 5 time steps for each trajectory.

## Network visualization

We provide tools for visualizing nodes of PBNs, trajectories and blocks.


In [ ]:
pbn.dependency_graph('dependency_graph', 'pdf', number_from_one=True) #Visualization of nodes and their mutual dependencies

In [ ]:
pbn.set_states([[True, True, True] for _ in range(512)], reset_history=True)
pbn.simple_steps(4)

for i in range(len(pbn.history_bool)):
    print(pbn.history_bool[i][0])

pbn.trajectory_graph(0, 'trajectory_binary_labels.svg', 'pdf') #Visualization of a given trajectory from simple_steps

## Decomposition of a Boolean Network into blocks

This is an example BN from the paper of dr Mizera, we will demonstrate library functionalities here.

There are six variables, $x_1,...,x_6$ with their boolean functions given as:

$$x_1: \lnot(x_1 \land x_2)$$

$$x_2: x_1 \land (\lnot x_2)$$

$$x_3:  \lnot x_2$$

$$x_4: (x_2 \land x_3) \lor x_5$$

$$x_5: x_4 \lor x_5$$

$$x_6: \lnot x_3 \lor x_6$$

In order to aid with efficient computation of attractors, a Boolean Network can be divided into *blocks*. A block is a concept defined on the dependencies between nodes, and it's closely related to the concept of a Strongly Connected Component. Every block is created by first dividing the network into SCCs, and then amending each SCC with the nodes which can influence it - that is, those that have in-edges to the SCC.

Below is a visual representation of the concept - SCCs on the left, and corresponding blocks on the right.
<div style="text-align: center;">
  <img src="./notebook_files/img/bn-mizera.png" alt="bnmiz" width="600"/>
</div>
<span style="font-size:60%">Source: "A new decomposition-based method for detecting attractors in synchronous Boolean networks."
Qixia Yuan, Andrzej Mizera, Jun Pang, and Hongyang Qu. Science of Computer Programming, 180:18-35, 2019. </span>

We will now showcase the functionality of decomposing the network into blocks on the exact Boolean Network shown in the image above.

In [ ]:
pbn = bang.load_from_file("examples/example.assa", "assa")
pbn.get_blocks()

These blocks directly correspond to the example pictured above, we have three two-element blocks and one four-element block.

We also have an option to generate a graph that visualizes the block decomposition of the loaded network.

In [ ]:
pbn.block_graph('block_graph', 'pdf', number_from_one=True) #Visualization of blocks in the network

## Attractor detection

We decompose Boolean Networks into smaller blocks with a subset of all nodes. Attractors computed on smaller blocks can be later combined to restore attractors of the original BN. In our library we implement algorithm from <br>

"*A new decomposition-based method for detecting attractors in synchronous Boolean networks.*" <br>
Qixia Yuan, Andrzej Mizera, Jun Pang, and Hongyang Qu. Science of Computer Programming, 180:18-35, 2019 <br>

by parallelizing block attractor detection on CPU via threads and single-block attractor detection on GPU with the use of simple_steps.

We showcase finding attractors in example BN using our divide and conquer-like implementation for Network from above:
<div style="text-align: center;">
  <img src="./notebook_files/img/bn-mizera.png" alt="bnmiz" width="500"/>
</div>

In [ ]:
import bang

example_bn = bang.PBN(
    6,                                                                      #Number of nodes of PBN
    [1, 1, 1, 1, 1, 1],                                                     #Number of Boolean functions per node
    [2, 2, 1, 3, 2, 2],                                                     #Number of variables per boolean function
    [                                                                       #Truth tables representing each function
        [True, True, True, False],
        [False, True, False, False],
        [True, False],
        [False, False, False, True, True, True, True, True],
        [False, True, True, True],
        [False, False, True, False],
    ],
    [[0, 1], [0, 1], [1], [1, 2, 4], [3, 4], [2, 5]],                       #Variables acting on each Boolean function
    [[1.0], [1.0], [1.0], [1.0], [1.0], [1.0]],                             #Probability of picking each function for different nodes (always one since we have BN)
    0.0,                                                                    #Probability of perturbation (0.0 since we have BN)
    [7],                                                                    #Nodes for which we dont perform perturbation (irrelevant since we have BN)
    update_type="synchronous"                                               #BN updates synchronously. All nodes update at once.
)

attractors = example_bn.blocks_detect_attractors()

for attractor in attractors:
    print(attractor)

## Monte Carlo attractor detection

In situations where simulating entire network is infeasible, especially for asynchronous networks, we can run Monte Carlo attractor detection. Our implementation utilizes fact that we can simulate PBN faster without saving trajectories. First we run long simple_step on multiple trajectories and afterwards we assume every state encountered now is a part of some attractor. We then run shorter simple_step with saving history and combine trajectories into separate attractors

In [ ]:
import bang

pbn = bang.load_from_file("examples/test2_no_perturbation.pbn", "assa", n_parallel=7)

attractors = pbn.monte_carlo_detect_attractors(trajectory_length=10000, attractor_length=100)

for attractor in attractors:
    print(attractor)

In [ ]:
#Block-based method returns the same attractors!
attractors = pbn.blocks_detect_attractors() 

for attractor in attractors:
    print(attractor)

## Benchmarks

We created a benchmarking suite to compare speed of PBN simulation on the GPU with a naive Python implementation and a CPU-based Numba-JIT implementation. Please note that the graph is in logarithmic scale.
<div style="text-align: center;">
  <img src="./notebook_files/img/gpu_cpu_comparison.png" alt="bnmiz" width="700"/>
</div>

As another benchmark, we checked if saving the current state at every step of GPU execution has an impact on performance.
<div style="text-align: center;">
  <img src="./notebook_files/img/save_history_comp.png" alt="bnmiz" width="700"/>
</div>

We also compared the GPU execution performance across update types.
<div style="text-align: center;">
  <img src="./notebook_files/img/update_types_comp.png" alt="bnmiz" width="700"/>
</div>

### Conclusions

We created Python library for modelling Boolean Networks which implements: <br>
- Network traversal with simple_step<br>
- Network visualization<br>
- Multiple methods of attractor detection<br>

**Thank You!**